In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy import special

In [2]:
def calc_expected_edges_p(n, p):
    if n < 2:
        raise ValueError("n muss größer als 1 sein!")
    if p < 0 or p > 1:
        raise ValueError("p muss Wahrscheinlichkeit sein!")
    
    return n-1 + p * ((n**2-3*n+2)/2)

In [3]:
def calc_number_edge_combis(n, e):
    s = 0
    for i in range(n):
        s += math.comb(e, i)
    return s

In [15]:
for i in [0,1,2,3,4,5,6,7,8,9,10]:
    print(calc_number_edge_combis(i, math.comb(i,2)))

0
1
2
7
42
386
4944
82160
1683218
40999516
1156626990


In [4]:
calc_number_edge_combis(10, math.comb(10,2))

1156626990

In [5]:
def load_optimum_tracking_files(p):
    """
    p ist pfad zu einer simulation_x
    
    """
    pt = os.path.join(p, "Tracking")
    pt_check = os.path.join(pt, "check")
#     print(pt_opt)
    
    df = pd.DataFrame()
    
    df['loop_counter'] = np.loadtxt(os.path.join(pt, "optimum_loop_counter.txt"), dtype=int)
    df['total[s]'] = np.loadtxt(os.path.join(pt, "optimum.txt"))
    check = [np.loadtxt(os.path.join(pt_check, i)) for i in sorted(os.listdir(pt_check), key=lambda x: int(x.split(".")[0]))]
    
    sums = [sum(i) for i in check]
    
    df['sum_check[s]'] = sums
#     d_f = {'counters': counters, 'setup_zeiten': setup, 'loop_zeiten': loop, 'total': total, 'opt': opt}
    
    return df

In [14]:
p1 = r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_8_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0"
a = load_optimum_tracking_files(p1)
a['anteil_check'] = a['sum_check[s]'] / a['total[s]']

In [15]:
# schonmal gut, dass dis sum_checks immer kleiner als die Gesamtlaufzeit ist
a[a['total[s]'] < a['sum_check[s]']]

,loop_counter,total[s],sum_check[s],anteil_check


In [16]:
a.mean()

loop_counter    4489.026000
total[s]           0.175228
sum_check[s]       0.073295
anteil_check       0.407823
dtype: float64

In [17]:
1 - (a.mean().loop_counter / calc_number_edge_combis(8, math.comb(8,2)))

0.9973330691568175

# Auswertung
Es ist so, dass hier, wenn ich einen Graph FC mit 8 Knoten nehme, dann im Schnitt 4489 (+1 wegen keine 1. stage Kanten) mal eine Kantenkombination checken muss. Das bedeutet, dass ich hier schon im mittel 99,73% aller möglichen Kombinationen abschneide.

### jetzt das TreePlusP pendant

In [20]:
# soviele Kanten hab ich wenn bei 8 Knoten FC:
print(calc_expected_edges_p(8, 1))
print(math.comb(8,2))

28.0
28


In [31]:
calc_expected_edges_p(10, 0.52778)

28.00008

Wenn ich 10 Knoten nehme und ein p von 0.52778, dann hab ich im Schnitt fast genau 28 Kanten, also genauso viele, wie im FC Fall.

## Überlegung:
Es ist aber so, dass ein Run mit 30 Kanten prozentual deutlich länger als ein 28 Kanten run dauert, als ein 26 Kanten run einspart. Das liegt daran, dass das Problem mit höherer Kantenzahl exponentiell schwerer wird, also kann man das so gar nicht vergleichen. Nur weil ich im Mittel die selbe Kantenanzahl habe, heißt das noch nicht, dass ich auch die selbe Laufzeit erwarte. Von diesem Run müsste ich eher alle runs mit genau 28 Kanten rausfiltern und nur für die vergleichen und da sollte aber dann das selbe rauskommen (stimmt auch nicht, siehe unten).

In [32]:
#lese die Ergebnisse ein und mache bool_mask wo nur die true sind, die genau 28 Kanten haben

In [6]:
# die Optimums maps einlesen
def readLGF_Network(source):
    mInFile = open(source, mode='r')
    fileString = mInFile.read()
    mInFile.close()
    if '@arcs' in fileString:
        initKey = '@arcs'
        initPos = 6
    elif '@edges' in fileString:
        initKey = '@edges'
        initPos = 7
    else:
        print('No keyword \'@arcs\' or \'@edges\' found\n Wrong file format')
        return
    strDat = fileString[fileString.find(initKey)+initPos:].split("\n", 1)[1]
    
    df = pd.DataFrame(columns=['u', 'v', 'id', 'first_stage'])
    
    for row in strDat.split('\n')[:-1]:
        s = row.split("\t")
        df = pd.concat([df,  pd.DataFrame({'u': int(s[0]), 'v': int(s[1]), 'id': int(s[2]), 'first_stage': int(s[3])}, index=[0])], ignore_index=True)
    return df

In [50]:
path_pendant = r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_pendant_10_nodes_5_scenarios\TreePlusP_0.527780_p_10_nodes_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\Optimum"
number_edges = [readLGF_Network(os.path.join(path_pendant, i)).shape[0] for i in sorted(os.listdir(path_pendant), key=lambda x: int(x.split(".")[0]))]

In [62]:
# jetzt die tracking daten einlesen und die number_edges als neue Spalte reinpacken
b = load_optimum_tracking_files(r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_pendant_10_nodes_5_scenarios\TreePlusP_0.527780_p_10_nodes_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0")
b['num_edges'] = number_edges 
b['anteil_check'] = b['sum_check[s]'] / b['total[s]']
b

,loop_counter,total[s],sum_check[s],num_edges,anteil_check
0,5955,0.251635,0.097666,29,0.388126
1,18523,0.629330,0.227317,30,0.361204
2,20960,0.712488,0.285068,32,0.400102
3,3207,0.116902,0.045768,30,0.391510
4,65982,3.883070,2.453197,32,0.631767
...,...,...,...,...,...
495,29524,1.334450,0.711461,24,0.533149
496,24179,0.936134,0.424877,26,0.453864
497,36876,1.406450,0.649360,31,0.461701
498,13510,0.684186,0.393363,28,0.574935


In [64]:
b_28_sub = b[b['num_edges']==28]
b_28_sub.mean()

loop_counter    34757.000000
total[s]            1.457896
sum_check[s]        0.727438
num_edges          28.000000
anteil_check        0.501203
dtype: float64

# Auswertung
Es ist so, dass hier, obwohl die Anzahl der Edges gleich ist, deutlich mehr Kombinationen angeschaut werden müssen und außerdem sind die optimalen Lösungen im Durchschnitt für das 'Pendant' teurer, nämlich 11.16, wohingegen, der Wert bei FC bei ca. 6.3 liegt:

In [77]:
opt_p10 = pd.read_csv(os.path.join(r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_pendant_10_nodes_5_scenarios\TreePlusP_0.527780_p_10_nodes_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0", "results.txt"), header=0, usecols=['Optimum'])
opt_p10['num_edges'] = np.loadtxt(os.path.join(r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_pendant_10_nodes_5_scenarios\TreePlusP_0.527780_p_10_nodes_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0", "edges_count.txt"), dtype=int)
(opt_p10[opt_p10['num_edges']==28]).mean()

Optimum      11.16035
num_edges    28.00000
dtype: float64

In [78]:
opt_fc8 = pd.read_csv(os.path.join(r"D:\Uni\Masterarbeit\Daten\Optimum_Tracking\Optimum_TimeTest_FC_8_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0", "results.txt"), header=0, usecols=['Optimum'])
opt_fc8.mean()

Optimum    6.29655
dtype: float64

In [97]:
(math.comb(28,8) + math.comb(28,9)) / calc_number_edge_combis(8, 28)

5.949915578374281

In [98]:
calc_number_edge_combis(10,28) / calc_number_edge_combis(8,28)

6.949915578374281

Es kommen über 5x soviele neue Kombinationen dazu, als es bei 8 FC insgesamt gibt

In [93]:
b_28_sub.mean()['total[s]'] / a.mean()['total[s]'] 

8.319988296266576

Wenn ich 10 Kantengraph mit 28 Kanten nehme, dann dauert eine Optimierung mehr als 8x so lange, wie bei 8 Knoten FC, also mit 28 Kanten. Da es 7x so viele Kombinationen zu überprüfen gibt, muss es aber noch einen anderen (kleinen) Teil des Algorithmus geben, der etwas länger braucht oder (also extra skaliert, vllt Kruskal?)? Weil eigentlich würde ich erwarten, dass der Algorithmus mit der Anzahl an Kantenkombinationen skaliert. Die Größenordnung ist zwar die selbe, aber trotzdem, will ich **das noch genauer untersuchen!!!**

Das liegt denke ich daran, dass ich bei FC weniger Kanten für einen Spannbaum brauche. Wenn ich 10 Knoten habe, dann brauche ich 9 Kanten für den Spannbaum und nicht nur 7. Da kommen nochmal 10Mio mögliche Kombinationen dazu. Die Problemstellung ist einfach eine andere. Prozentual gesehen, werden ungefähr gleich viele Kombinationen weggeschnitten:

In [79]:
# halt: ich brauche ja auch mehr Kanten für einen Spannbaum
1-(b_28_sub.mean().loop_counter / calc_number_edge_combis(10, 28))

0.997028864982314

# Jetzt meine aktuelle Verbesserung
Da will ich jetzt wissen, wie sehr verbessert das nochmal die Laufzeit (in Kombination mit der bisherigen Optimierung). Ich muss also verschiedene Problemstellungen simulieren und zwar einmal mit dem alten bruteforce Algorithmus und einmal mit dem neuen. Am besten über die exakt selben Problemstellungen, damit ich wirklich einen Unterschied bei den selben Bedingungen sehen kann.

Außerdem will ich den obigen Test für den LP-Alg. wiederholen. Also schauen, ob der LP-Alg. auch für FC schneller ist, als für TreePlusP mit der selben Kantenanzahl wie der FC Graph.

Und ich will nach der Optimierung vom optimalen Alg timen, wie die Zeitunterschied zwischen LP-Approx und Optimalem Alg ist.

### Zuerst schauen, ob die selben Ergebnisse rauskommen

In [7]:
def read_files(konfig_path):
    """
    Ist dafuer da, fuer eine Konfiguration alle Simulationen einzulesen und in ein Dataframe zu packen
    """
    
    # muss erstmal rausfinden, welche Simulationen alle gemacht wurden
    sims = os.listdir(konfig_path)
#     print(sims)

    # erstes Dataframe, an das dann angebaut wird
    df = pd.read_csv(os.path.join(konfig_path, sims[0], "results.txt")).iloc[:,:-1]
    
    for sim in sims[1:]:
#         print(sim)
#         # nehme nur alle Ordner und keine Datein
#         for alg in [a for a in os.listdir(os.path.join(dir_path, sim)) if not "." in a]:
#             print(alg)

        # nehme die results und packe sie an df dran
        df_sim = pd.read_csv(os.path.join(konfig_path, sim, "results.txt")).iloc[:,:-1]
        # dabei gehe ich aus, dass selbst, wenn die results in unterschiedlichen Spalten gespeichert werden, dass
        # die an die dazugehoerige, bereits vorhandene Spalte angegliedert werden
        df = pd.concat([df, df_sim], ignore_index=True)
    
    return df

In [103]:
ovo_res = read_files(r"D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum8_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000")
ovo_res[ovo_res["Optimum"]!=ovo_res['Optimum2']]

,Optimum,Optimum2


Es kommt also immer das selbe raus. Das ist schon mal sehr gut. **Jetzt will ich die Laufzeiten und Anzahl an gecheckten Edge-Kombinationen vergleichen**

In [8]:
def load_ovo_tracking_files(p):
    """
    p ist pfad zu einer simulation_x
    
    """
    pt = os.path.join(p, "Tracking")
    pt_check = os.path.join(pt, "check")
    pt_check_neu = os.path.join(pt, "check2")
#     print(pt_opt)
    
    df_alt = pd.DataFrame()
    
    df_alt['loop_counter'] = np.loadtxt(os.path.join(pt, "optimum_loop_counter.txt"), dtype=int)
    df_alt['total[s]'] = np.loadtxt(os.path.join(pt, "optimum.txt"))
    check_alt = [np.loadtxt(os.path.join(pt_check, i)) for i in sorted(os.listdir(pt_check), key=lambda x: int(x.split(".")[0]))]
    
    sums_alt = [sum(i) for i in check_alt]
    
    df_alt['sum_check[s]'] = sums_alt
#     d_f = {'counters': counters, 'setup_zeiten': setup, 'loop_zeiten': loop, 'total': total, 'opt': opt}
    
    df_neu = pd.DataFrame()
    
    df_neu['loop_counter'] = np.loadtxt(os.path.join(pt, "optimum_loop_counter2.txt"), dtype=int)
    df_neu['total[s]'] = np.loadtxt(os.path.join(pt, "optimum2.txt"))
    check_neu = [np.loadtxt(os.path.join(pt_check_neu, i)) for i in sorted(os.listdir(pt_check_neu), key=lambda x: int(x.split(".")[0]))]
    
    sums_neu = [sum(i) for i in check_neu]
    
    df_neu['sum_check[s]'] = sums_neu
    
    return df_alt, df_neu

In [109]:
df_alt, df_neu = load_ovo_tracking_files(r"D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum8_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0")

In [112]:
df_alt['anteil_check'] = df_alt['sum_check[s]'] / df_alt['total[s]']
df_neu['anteil_check'] = df_neu['sum_check[s]'] / df_neu['total[s]']
df_alt.mean()

loop_counter    4329.782000
total[s]           0.229405
sum_check[s]       0.083641
anteil_check       0.340399
dtype: float64

In [113]:
df_neu.mean()

loop_counter    1547.486000
total[s]           0.143267
sum_check[s]       0.062309
anteil_check       0.428800
dtype: float64

In [115]:
1 - (df_neu.mean().loop_counter / calc_number_edge_combis(8, 28))

0.9990806383962149

In [121]:
math.comb(4,2)

6

In [122]:
calc_number_edge_combis(4, 6)

42

## Jetzt 11 Knoten FC
Das Einlesen der Daten dauert ewig (wegen check, weil ich so viele Iterationen habe)

In [9]:
df_alt11, df_neu11 = load_ovo_tracking_files(r"D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum11_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0")

Wie viele moeglichen Erstphasenselektionen gibt es?

Hab hier 500 runs

In [44]:
dfe11 = pd.DataFrame()
dfe11['loop_counter'] = np.loadtxt(os.path.join(r'D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum11_nodes_5_scenarios', "optimum_loop_counter2.txt"), dtype=int)

In [46]:
(1 - dfe11['loop_counter'] / calc_number_edge_combis(11, math.comb(11,2))).mean() * 100

99.99981871542896

In [54]:
(1 - df11c['loop_counter'] / calc_number_edge_combis(11, math.comb(11,2))).mean() * 100

99.99981876782643

In [52]:
df11c = pd.concat([df_neu11, dfe11], ignore_index=True)

In [53]:
((1-(df11c['loop_counter'] / calc_number_edge_combis(11, math.comb(11,2))))*100).std()

0.0002213741879099331

In [55]:
99.99981876782643 + 0.0002213741879099331

100.00004014201434

99.9998(2)

In [23]:
math.comb(15,2)

105

In [10]:
calc_number_edge_combis_edge_combis(11, math.comb(11,2))

37060382822

In [124]:
df_alt11['anteil_check'] = df_alt11['sum_check[s]'] / df_alt11['total[s]']
df_neu11['anteil_check'] = df_neu11['sum_check[s]'] / df_neu11['total[s]']
df_alt11.mean()

loop_counter    180573.592000
total[s]             8.933071
sum_check[s]         4.110229
anteil_check         0.447276
dtype: float64

In [125]:
df_neu11.mean()

loop_counter    67126.500000
total[s]            5.137850
sum_check[s]        3.215376
anteil_check        0.616012
dtype: float64

In [133]:
(1 - (df_alt11.mean().loop_counter / calc_number_edge_combis(11, math.comb(11,2)))) * 100

99.9995127584276

In [132]:
(1 - (df_neu11.mean().loop_counter / calc_number_edge_combis(11, math.comb(11,2)))) * 100

99.99981887262113

In [130]:
(df_alt11.mean()['total[s]'] - df_neu11.mean()['total[s]']) / df_alt11.mean()['total[s]']

0.4248506211656029

Ok, beide Versionen schneiden einen sehr großen Anteil an Kantenkombinationen weg und die neue Version ist nochmal besser und braucht 43% weniger Zeit, als die bisherige Version. Das ist schon echt nicht schlecht. So kann ich dieses Problem mit 11 Knoten Fc, also 55 Kanten mit 5 Szenarien und random Kosten zwischen 0 und 10 in etwas über 5s lösen. Außerdem geht ein deutlich größerer Teil der Gesamtlaufzeit bei der optimierten Version für die check-Aufrufe drauf. Das bedeutet, dass ich bei dieser Version weniger Zeit außerhalb dieser Aufrufe verbringe.

In [134]:
math.comb(15,2)

105

Jetzt mit 15 Knoten, aber da sind die check_Ordner so groß geworden, dass ich sie weglasse

In [4]:
def load_ovo_tracking_files_no_check(p):
    """
    p ist pfad zu einer simulation_x
    
    """
    pt = os.path.join(p, "Tracking")

#     print(pt_opt)
    
    df_alt = pd.DataFrame()
    
    df_alt['loop_counter'] = np.loadtxt(os.path.join(pt, "optimum_loop_counter.txt"), dtype=int)
    df_alt['total[s]'] = np.loadtxt(os.path.join(pt, "optimum.txt"))
#     check_alt = [np.loadtxt(os.path.join(pt_check, i)) for i in sorted(os.listdir(pt_check), key=lambda x: int(x.split(".")[0]))]
    
#     sums_alt = [sum(i) for i in check_alt]
    
#     df_alt['sum_check[s]'] = sums_alt
#     d_f = {'counters': counters, 'setup_zeiten': setup, 'loop_zeiten': loop, 'total': total, 'opt': opt}
    
    df_neu = pd.DataFrame()
    
    df_neu['loop_counter'] = np.loadtxt(os.path.join(pt, "optimum_loop_counter2.txt"), dtype=int)
    df_neu['total[s]'] = np.loadtxt(os.path.join(pt, "optimum2.txt"))
#     check_neu = [np.loadtxt(os.path.join(pt_check_neu, i)) for i in sorted(os.listdir(pt_check_neu), key=lambda x: int(x.split(".")[0]))]
    
#     sums_neu = [sum(i) for i in check_neu]
    
#     df_neu['sum_check[s]'] = sums_neu
    
    return df_alt, df_neu

In [5]:
dfe15 = pd.DataFrame()
dfe15['loop_counter'] = np.loadtxt(os.path.join(r'D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum15_nodes_5_scenarios', "optimum_loop_counter2.txt"), dtype=int)

In [6]:
df_alt15, df_neu15 = load_ovo_tracking_files_no_check(r"D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0")
df_alt15.mean()

loop_counter    1.912509e+07
total[s]        9.915773e+02
dtype: float64

In [11]:
df15c = pd.concat([df_neu15, dfe15], ignore_index=True)

In [12]:
(1 - df15c['loop_counter'] / calc_number_edge_combis(15, math.comb(15,2))).mean() * 100

99.99999999208256

In [13]:
((1-(df15c['loop_counter'] / calc_number_edge_combis(15, math.comb(15,2))))*100).std()

1.0770157896818393e-08

In [14]:
0.00000001

1e-08

In [35]:
((1-(df_neu15['loop_counter'] / calc_number_edge_combis(15, math.comb(15,2))))*100).std()

7.467179534567478e-09

In [37]:
0.000000007

7e-09

In [149]:
df_alt15.mean().loop_counter / df_neu15.mean().loop_counter

2.6104211005672577

In [10]:
(1 - (df_alt15.mean().loop_counter / calc_number_edge_combis(15, math.comb(15,2)))) * 100

99.99999998228229

In [27]:
(1 - (df_neu15.mean().loop_counter / calc_number_edge_combis(15, math.comb(15,2)))) * 100

99.9999999932127

In [41]:
99.9999999932127 + 7.467179534567478e-09

100.00000000067988

In [141]:
(df_alt15.mean()['total[s]'] - df_neu15.mean()['total[s]']) / df_alt15.mean()['total[s]']

0.3578855558287901

Es ist anscheinend so, dass ich je mehr Möglichkeiten es gibt, auch prozentual gesehen immer mehr davon wegschneide, das geht auch exponentiell, schafft es aber nicht das einzuholen. Der Check_Zeit anteil ist vllt. doch interessant

In [164]:
def read_long_check_files(p_check_neu, p_save):
    # ist gedacht zum nur 1x ausfuehren
    
#     sums_alt = []
    sums_neu = []
    
#     for f1 in os.listdir(p_check_alt):
#         a = np.loadtxt(os.path.join(p_check_alt, f1))
#         sums_alt.append(a.sum())
    for f2 in os.listdir(p_check_neu):
        a = np.loadtxt(os.path.join(p_check_neu, f2))
        sums_neu.append(a.sum())
    
    # speichern und returnen
#     np.savetxt(os.path.join(p_save, "summe_alt_s.txt"), sums_alt)
    np.savetxt(os.path.join(p_save, "summe_neu_s.txt"), sums_neu)
    
    return sums_neu

In [166]:
p_check_alt = r"\\?\D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\Tracking\check"
p_check_neu = r"\\?\D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\Tracking\check2"
p_save = r"D:\Uni\Masterarbeit\Daten\optimum_vs_optimum\OptimumVSOptimum15_nodes_5_scenarios\FullyConnected_5_RandomScenarioCreator_RandomTestCreator_0.000000_10.000000\simulation_0\Tracking"
sn = read_long_check_files(p_check_neu, p_save)

In [220]:
# muss die listen sa und sn umsortieren
arr = list(np.arange(50))
arra=list(np.arange(51))
iis = [pair[0] for pair in sorted(zip(arr, os.listdir(p_check_neu)), key=lambda n: int(n[1].split(".")[0]))]
iisa = [pair[0] for pair in sorted(zip(arra, os.listdir(p_check_alt)), key=lambda n: int(n[1].split(".")[0]))]
# iisa

In [217]:
df_alt15['sum_check[s]'] = (np.array(sa)[iisa])[:-1]

In [208]:
df_neu15['sum_check[s]'] = np.array(sn)[iis]

In [221]:
# so, jetzt die Verhaeltnisse ausrechnen
df_alt15['anteil_check'] = df_alt15['sum_check[s]'] / df_alt15['total[s]']
df_neu15['anteil_check'] = df_neu15['sum_check[s]'] / df_neu15['total[s]']

In [224]:
print(df_alt15.mean().anteil_check)
print(df_neu15.mean().anteil_check)

0.5644053556522841
0.7177991458369624
